In [1]:
# Notebook 3 — Friction & Power Usage
import pandas as pd
import numpy as np

transfers = pd.read_csv("../data/transfers_level.csv", parse_dates=["created_dt","funded_dt","transferred_dt"])

# Time deltas (friction)
transfers["to_funded_hours"] = (transfers["funded_dt"] - transfers["created_dt"]).dt.total_seconds() / 3600
transfers["to_transferred_hours"] = (transfers["transferred_dt"] - transfers["funded_dt"]).dt.total_seconds() / 3600

by_seg = ["region","platform","experience"]
friction = transfers.groupby(by_seg, dropna=False).agg(
    created=("transfer_created","sum"),
    funded=("transfer_funded","sum"),
    transferred=("transfer_transferred","sum"),
    ttf_median_hours=("to_funded_hours","median"),
    ttt_median_hours=("to_transferred_hours","median")
).reset_index()

friction["cr_created_to_funded"] = (friction["funded"]/friction["created"]).fillna(0)
friction["cr_funded_to_transferred"] = (friction["transferred"]/friction["funded"]).fillna(0)
friction["failure_rate"] = (friction["created"] - friction["transferred"]).div(friction["created"]).fillna(0)
friction["success_rate"] = 1 - friction["failure_rate"]

friction.to_csv("../data/friction_segmented.csv", index=False)

# Power usage: user-level intensity vs success
user_stats = transfers.groupby("user_id").agg(
    transfers_created=("transfer_created","sum"),
    transfers_transferred=("transfer_transferred","sum")
).reset_index()
user_stats["success_rate"] = (user_stats["transfers_transferred"] / user_stats["transfers_created"]).fillna(0)
user_stats["intensity"] = user_stats["transfers_created"]

bins = [0,1,2,5,10,25,50,1e9]
labels = ["1","2","3-5","6-10","11-25","26-50","50+"]
user_stats["intensity_bucket"] = pd.cut(user_stats["intensity"], bins=bins, labels=labels, right=True)

power = user_stats.groupby("intensity_bucket").agg(
    users=("user_id","count"),
    avg_success_rate=("success_rate","mean"),
    total_transfers=("intensity","sum")
).reset_index()

power.to_csv("../data/power_usage.csv", index=False)

print("Friction and power usage tables exported.")

Friction and power usage tables exported.


/var/folders/ml/klt3p20j5xj8cgxtrxqrd6s00000gn/T/ipykernel_1927/246389249.py:39: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  power = user_stats.groupby("intensity_bucket").agg(
